# ag2gcal - Texas A&M Planting Calendar to gCal

Convert the HTML calendars from the [Aggie Horticulture (R)](https://aggie-horticulture.tamu.edu/) site at `calendar_source_urls` to Google Calendar event entries at `calendar_destination_ids`.

Disclaimers:

- I am not an Aggie.
- Aggie Horticulture (R) is not my registered trademark.
- Thank you to the Aggie Horticulture team for providing these super helpful calendars. All credit to them for crafting these.

Example:

![](img/ag2gcal.png)

In [25]:
import datetime
from datetime import date
from datetime import timedelta
import os
import base64
import time

import pandas as pd
import dateparser
import gcsa
import gcsa.recurrence

from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar

### Various variables

- `region_columns` - list of all regions contained in the tables
- `all_columns` - includes the leftmost index for the veggie
- `gcloud_creds_path` - OAuth credentials with permissions to access the person's calendars
- `calendar_source_urls` - source sites for the planting schedules
- `calendar_destination_ids` - map of region to gCal calendar ID where we'll eventually write the calendar events

In [15]:
region_columns = ['Region I', 'Region II', 'Region III', 'Region IV', 'Region V']
all_columns = ['Vegetable'] + region_columns
start_of_this_year = date(date.today().year, 1, 1)
start_of_next_year = date(date.today().year + 1, 1, 1)

### Google Cloud Credentials

1. Open `Google Cloud Console -> IAM & Admin -> Service Accounts`
2. Add a new service account
3. Create a new key and download it in the `JSON` format
4. Store in a secure place on your machine
5. Set an environment variable called `GOOGLE_CLOUD_AG2GCAL_CREDS_PATH` with the full path to your credentials file

### Google Cloud Credentials

1. Open `Google Cloud Console -> IAM & Admin -> Service Accounts`
2. Add a new service account
3. Create a new key and download it in the `JSON` format
4. Store in a secure place on your machine
5. Set an environment variable called `GOOGLE_CLOUD_AG2GCAL_CREDS_PATH` with the full path to your credentials file

In [4]:
gcloud_creds_path = os.environ['GOOGLE_CLOUD_AG2GCAL_CREDS_PATH']

### Source Calendars

The source calendars from the Aggie Horticulture site.

In [5]:
calendar_source_urls = {
    'Spring': 'https://aggie-horticulture.tamu.edu/archives/parsons/earthkind/ekgarden14.html',
    'Fall': 'https://aggie-horticulture.tamu.edu/archives/parsons/fallgarden/falldirect.html'
}

### Destination Calendars

A dictionary of (Region, Google Calendar ID) in which to dump the calendar events we scrape.

In [6]:
calendar_destination_ids = {
    'Region III': 'c_4kikah5kfau07c7v9t5fieent0@group.calendar.google.com'
}

### Parse date ranges from tables

The dates in the tables are somewhat inconsistent. Play around with them a bit to get them into a useable, two entry array of `[start_date, end_date]`.

In [7]:
def try_parse_date_range(val, year=datetime.datetime.now().year, default_window_in_days=14, date_format='%b %d %Y'):
    stringified_val = str(val)
    
    start_date = None
    end_date = None
    
    if '-' in stringified_val:
        raw_date_range = val.split('-')
        
        if len(raw_date_range) == 2:
            raw_date_range[:] = [f'{raw_date} {year}' for raw_date in raw_date_range]
            
            start_date = raw_date_range[0]
            end_date = raw_date_range[1]
    elif stringified_val.startswith('After'):
        start_date = f'{stringified_val.replace("After", "")} {year}'
    else:
        start_date = f'{stringified_val} {year}'
        
    if start_date != "nat" and isinstance(start_date, str):
        start_date = dateparser.parse(start_date)
        
        if end_date != None:
            end_date = dateparser.parse(end_date)
        
        if start_date != None and end_date == None:
            end_date = start_date + timedelta(days=default_window_in_days)
    
    return [start_date, end_date]
    

### Parse the HTML tables and coerce them in to DataFrames

In [8]:
def parse_url(season, url):
    print(url)
    
    raw_tables = pd.read_html(url)
    
    calendar_table = raw_tables[1]
    
    if calendar_table.iloc[0][0] == 'Vegetables':
        calendar_table.drop([0], inplace=True)

    calendar_table.columns = all_columns
    calendar_table['season'] = season
        
    return calendar_table

### Do it

In [9]:
raw_calendars = pd.DataFrame()

for calendar_source in calendar_source_urls.items():
    calendar = parse_url(*calendar_source)
    raw_calendars = raw_calendars.append(calendar)
    
raw_calendars.reset_index(inplace=True)

https://aggie-horticulture.tamu.edu/archives/parsons/earthkind/ekgarden14.html
https://aggie-horticulture.tamu.edu/archives/parsons/fallgarden/falldirect.html


In [10]:
for region_column in region_columns:
    raw_calendars[region_column] = raw_calendars[region_column].apply(try_parse_date_range)

In [11]:
unpivoted_calendar = raw_calendars.melt(id_vars=['Vegetable', 'season'], value_vars=region_columns, var_name='region', value_name='date_range')

In [12]:
unpivoted_calendar['start_date'] = unpivoted_calendar.apply(lambda row: row['date_range'][0], axis=1)
unpivoted_calendar['end_date'] = unpivoted_calendar.apply(lambda row: row['date_range'][1], axis=1)
unpivoted_calendar.drop(['date_range'], axis=1, inplace=True)

### Establish gcal clients

This uses a desktop OAuth flow, so each calendar will kick out a URL to authorize. You'll need to replace the `&amp;` in the URL with actual ampersands.

If you run this a second time after your token has expired, you'll need to delete the `token.pickle` file that GCSA creates and try to re-auth. Otherwise, you get a funky error about scopes.

TODO: figure out how to output and not urlencode within a given cell.

In [ ]:
calendar_clients = {}

for region, calendar_id in calendar_destination_ids.items():
    calendar_clients[region] = GoogleCalendar(calendar_id, credentials_path=gcloud_creds_path)

### HACK - Clear out old events

I couldn't find a stable way to predict the calendar event ID nor how to store some metadata in the calendar event that would allow us to deterministically find it later. Instead, we just delete everything to keep the calendar clean, yay!

In [31]:
all_events = {}

rate_limit_counter = 0
rate_limit_after = 25

for region, client in calendar_clients.items():
    print(f'clearing out old events for {region}')
    
    for event in client[start_of_this_year:start_of_next_year:'updated']:
        client.delete_event(event)
        print('☠️', end='')
        
        rate_limit_counter += 1
        
        if rate_limit_counter >= rate_limit_after:
            print('😴')
            rate_limit_counter = 0
            time.sleep(1)
            
print('✅')

clearing out old events for Region III
☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️😴
☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️😴
☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️☠️✅


### Write events to calendars

Iterate over each of the regions and write the events for that region to the respective calendar defined in `calendar_source_urls`

In [32]:
for region in region_columns:
    print(f'Adding events for {region}')
    
    if region in calendar_clients:
        calendar_client = calendar_clients[region]
    
        for index, row in unpivoted_calendar[unpivoted_calendar['region'] == region].iterrows():
            if row['start_date'] != None and not pd.isnull(row['start_date']):
                humanized_event_id = f'{row["season"]}/{row["Vegetable"]}'

                event = Event(
                    row['Vegetable'],
                    start=row['start_date'],
                    end=row['end_date']
                )

                create_results = calendar_client.add_event(event)
                
                print(f'✅ {humanized_event_id}')

print('🎉🎉🎉')

Adding events for Region I
Adding events for Region II
Adding events for Region III
✅ Spring/Asparagus
✅ Spring/Beans, snap bush
✅ Spring/Beans, snap pole
✅ Spring/Beans, Lima bush
✅ Spring/Beans, Lima pole
✅ Spring/Beets
✅ Spring/Broccoli
✅ Spring/Cabbage
✅ Spring/Carrots
✅ Spring/Cauliflower (transplant)
✅ Spring/Chard, Swiss
✅ Spring/Collards Mar 1-May 1Feb 15-Apr 10Feb 1-Mar 25Jan 15-Mar 15Jan 1-Mar 15
✅ Spring/Corn, sweet
✅ Spring/Cucumber
✅ Spring/Eggplant
✅ Spring/Kohlrabi
✅ Spring/Lettuce
✅ Spring/Muskmelon (cantaloupe)
✅ Spring/Mustard
✅ Spring/Onion (plants)
✅ Spring/Parsley
✅ Spring/Peas, English
✅ Spring/Peas, Southern
✅ Spring/Pepper (transplant)
✅ Spring/Potato
✅ Spring/Potato, sweet (slips)
✅ Spring/Pumpkin
✅ Spring/Radish
✅ Spring/Spinach
✅ Spring/Squash, summer
✅ Spring/Squash, winter
✅ Spring/Tomato (transplant)
✅ Spring/Turnip
✅ Spring/Watermelon
✅ Fall/Beans, snap bush
✅ Fall/Beans, Lima bush
✅ Fall/Beets
✅ Fall/Broccoli
✅ Fall/Brussels sprouts
✅ Fall/Cabbage
✅ Fall